In [25]:
import pandas as pd
import torch
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from sklearn.model_selection import GroupKFold
from torch_geometric.data import InMemoryDataset
import networkx as nx
import os
from scipy import signal

import torch
from torch_geometric.data import InMemoryDataset, download_url

In [12]:
test_patients = set(['F117', 'F130', 'F128', 'F133', 'F135', 'F138', 'F139', 'F136',
                 'F131', 'F344', 'F504', 'F510', 'M512', 'F341', 'F337', 'F324',
                 'F308', 'F317'])

In [17]:
# unq_set = set()
# for elem in os.listdir('clean_wcoh/kz_clean/health/'):
#     if 'log' in elem:
#         continue
#     name = elem.split('_')[1]
#     if name in test_patients:
#         if name not in unq_set:
#             unq_set.add(name)

In [28]:
elecs = ['Fp1', 'Fp2', 'Fpz', 'F3', 'F4', 'Fz', 'C3', 'C4',
         'Cz', 'P3', 'P4', 'Pz', '01', '02', '0z', 'F7', 
         'F8', 'T3', 'T4','T5', 'T6']

In [26]:
adj = np.load('clean_wcoh/kz_clean/health/clean_F116_patch_1.npy')

In [72]:
def vectorize_adj_mat_coo(matrix, allow_loops = True):
    source_nodes = []
    target_nodes = []
    if allow_loops:
        for i in range(matrix.shape[0]):
            for j in range(i, matrix.shape[1]):
                source_nodes.append(i)
                target_nodes.append(j)
    else:
        for i in range(matrix.shape[0]):
            for j in range(i + 1, matrix.shape[1]):
                source_nodes.append(i)
                target_nodes.append(j)
    return source_nodes, target_nodes

In [65]:
def vectorize_adj_mat_weights(matrix, allow_loops = True, weighted = True, threshold = 0.5):
    edge_weights = []
    if weighted:
        if allow_loops:
            for i in range(matrix.shape[0]):
                for j in range(i, matrix.shape[1]):
                    edge_weights.append(matrix[i][j])
        else:
            for i in range(matrix.shape[0]):
                for j in range(i + 1, matrix.shape[1]):
                    edge_weights.append(matrix[i][j])
    else:
        mask = np.array((matrix > threshold), dtype = np.uint8)
        if allow_loops:
            for i in range(mask.shape[0]):
                for j in range(i, mask.shape[1]):
                    edge_weights.append(mask[i][j])
        else:
            for i in range(mask.shape[0]):
                for j in range(i + 1, mask.shape[1]):
                    edge_weights.append(mask[i][j])
    return edge_weights

In [ ]:
class GNNDataset(InMemoryDataset):
#     def __init__(self, root_raw, root_spectrum,file_path,ids, transform=None, pre_transform=None):
#         self.df_part1 =pd.read_csv(file_path)
#         self.root_raw =root_raw
#         self.root_spectrum =root_spectrum
#         self.ids=ids
#         super(GNNDataset, self).__init__(root_raw, transform,pre_transform)
#         self.data, self.slices = torch.load(self.processed_paths[self.ids])
        
    
    def __init__(self, root, idx, feature_names, allow_loops = True, weighted = True, threshold = 0.6,
                 transform = None, pre_transform = None, pre_filter = None):
        self.data = np.load(root)
        self.stage = idx # idx 0 - train, idx 1 - test
        self.feature_names = feature_names
#         self.offset = 0 пока хз
        self.allow_loops = allow_loops
        self.weighted = weighted
        self.threshold = threshold
        
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[self.stage])
        
#     def get_eeg_dataset(self,eeg_raw, raw_path, slice_path,slice_size =21500):
#         slice_csv =slice_path / eeg_raw['slice_id']
#         slice1 =pd.read_csv(slice_csv)
#         file_name = raw_path / eeg_raw['fn']
#         eeg =pd.read_csv(file_name).iloc[slice_size*int(eeg_raw['start']): slice_size*(int(eeg_raw['start'])+1), :]
#         eeg =eeg.drop( columns =['Unnamed: 0', 'time'])
#         return slice1, eeg
    
#     def extract_features(self,eeg):
#         df_bands =pd.DataFrame(pd.Series(get_bands_feats(eeg))).reset_index()
#         df_bands["electrode"] =df_bands["index"].str.split("_").apply(lambda x :x[-1])
#         df_bands["band"] =df_bands["index"].str.split("_").apply(lambda x :x[-2])
#         return df_bands.pivot(index='band', columns ='electrode', values=0).reindex(columns=eeg.columns)

#     def get_eeg_data(self,slice1, eeg,label, treshold =0.5):
#         columns_dict = {i : column for i, column in enumerate(eeg.columns.tolist())}
#         columns_dict_inverted ={column : i for i, column in enumerate(eeg.columns.tolist())}
#         X=self.extract_features(eeg).T.to_numpy()
#         edges,edge_attrs =[], []
#         for raw in slice1.iterrows():
#             edge =raw[1]['Unnamed: 0'].split("_")
#             edge[0]=columns_dict_inverted[edge[0]]
#             edge[1]=columns_dict_inverted[edge[1]]
#             edge_attr =raw[1]['0']
#             if edge_attr>treshold:
#                 edges.append(edge.copy())
#                 edge_attrs.append(edge_attr)
#         return Data(x=torch.tensor(X), edge_index=torch.tensor(np.array(edges).T),edge_attrs=edge_attrs, y=torch.tensor([label]))  
    
    def vectorize_adj_mat_coo(self, matrix, allow_loops = True):
        source_nodes = []
        target_nodes = []
        if allow_loops:
            for i in range(matrix.shape[0]):
                for j in range(i, matrix.shape[1]):
                    source_nodes.append(i)
                    target_nodes.append(j)
                    edge_weights.append(matrix[i][j])
        else:
            for i in range(matrix.shape[0]):
                for j in range(i + 1, matrix.shape[1]):
                    source_nodes.append(i)
                    target_nodes.append(j)
                    edge_weights.append(matrix[i][j])
        return source_nodes, target_nodes
    
    def vectorize_adj_mat_weights(self, matrix, allow_loops = True, weighted = True, threshold = 0.5):
        edge_weights = []
        if weighted:
            if allow_loops:
                for i in range(matrix.shape[0]):
                    for j in range(i, matrix.shape[1]):
                        edge_weights.append(matrix[i][j])
            else:
                for i in range(matrix.shape[0]):
                    for j in range(i + 1, matrix.shape[1]):
                        edge_weights.append(matrix[i][j])
        else:
            mask = np.array((matrix > threshold), dtype = np.uint8)
            if allow_loops:
                for i in range(mask.shape[0]):
                    for j in range(i, mask.shape[1]):
                        edge_weights.append(mask[i][j])
            else:
                for i in range(mask.shape[0]):
                    for j in range(i + 1, mask.shape[1]):
                        edge_weights.append(mask[i][j])
        return edge_weights

    ## TODO and check
    
    def upload_data(self, patch_name): #, slice_length = 21): #path_to_data, path_to_adj_matr, patch_name, slice_length = 21):
        '''
        input:
            path_to_data: path to precomputed node representations
            path_to_adj_matr: path to precomputed adj matrices
            
        returns:
            Pygeometric Data object (see PyG docs)
        '''
#         eeg = np.load(path_to_data)[self.offset : self.offset + slice_length, :]
        X, target, adj_matrix = self.data[patch_name] # triplet in format [X, target, A]
#         self.offset += slice_length
#         adj_matrix = np.load()
        edge_index = np.array(self.vectorize_adj_mat_coo(adj_matrix, 
                                                         allow_loops = self.allow_loops))
        edge_features = self.vectorize_adj_mat_weights(adj_matrix, 
                                                       allow_loops = self.allow_loops,
                                                       weighted = self.weighted, 
                                                       treshold = self.threshold)
        return Data(x = torch.tensor(X), 
                    edge_index = torch.tensor(edge_index),
                    edge_attrs = edge_features, 
                    y = torch.tensor([target]))  
    
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return [f'gnn_dataset_train_{self.feature_names}.pt',
                f'gnn_dataset_test_{self.feature_names}.pt']

    def download(self):
        pass
    
#     def process(self):
#         print('process')
#         print(self.df_part1)
#         data_list = []
#         for ind in range(self.df_part1.shape[0]):
#             slice1, eeg=self.get_eeg_dataset(self.df_part1.iloc[ind], Path(self.root_raw), Path(self.root_spectrum))
#             data_exmp =self.get_eeg_data(slice1, eeg, self.df_part1.iloc[ind]['target'], 0.5)
#             data_list.append(data_exmp)
            
        
#         data, slices = self.collate(data_list)
#         print(data)
#         torch.save((data, slices), self.processed_paths[self.ids])
        
    def process(self):
#         print('process')
#         print(self.df_part1)
        data_list = []
        for elem in self.data.keys():
            data_list.append(self.upload_data(elem))
#         for ind in range(self.df_part1.shape[0]):
#             slice1, eeg=self.get_eeg_dataset(self.df_part1.iloc[ind], Path(self.root_raw), Path(self.root_spectrum))
#             data_exmp =self.get_eeg_data(slice1, eeg, self.df_part1.iloc[ind]['target'], 0.5)
#             data_list.append(data_exmp)
            
        
        data, slices = self.collate(data_list)
#         print(data)
        torch.save((data, slices), self.processed_paths[self.stage])
    

In [24]:
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform = None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        # Download to `self.raw_dir`.
        return
    
    def process(self):
        # Read data into huge `Data` list.
        data_list = [...]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])